In [1]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
from tabulate import tabulate

# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential3.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [13]:
theme_lookup = {"01": "Points", "02": "Lines", "03": "Polygons", "04": "Hexagons", "05": "Blue", "06": "Red", "07": "Green", "08": "Yellow", "09": "Monochrome", "10": "Grid", "11": "3D", "12": "Map not made with GIS software", "13": "Raster", "14": "Climate change", "15": "Connections", "16": "Island(s)", "17": "Historical map", "18": "Landuse", "19": "NULL", "20": "Population", "21": "Water", "22": "Movement", "23": "Boundaries", "24": "Elevation", "25": "COVID-19", "26": "Map with a new tool", "27": "Big or small data", "28": "Non-geographic map", "29": "Globe", "30": "A map"}

In [14]:
theme_lookup["01"]

'Points'

#### Set date

In [2]:
date = '2020-11-05'
start = '2020-11-05 00:00:00'
end = '2020-11-06 00:00:00'

In [93]:
# date = '2020-11-03'
# df = pd.read_csv("./data/{}.csv".format(date))
# print("{}.csv".format(date))

2020-11-03.csv


#### Collect tweets with hashtag `#30mapchallenges`

In [78]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet':retweet_count,
                        'favorite':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [79]:
search_words = "#30DayMapChallenge -filter:retweets"
since = date

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= start) & 
   (df.tweet_created_at <= end)]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


df.to_csv("./data/{}.csv".format(date))

In [80]:
df

,tweet_id,tweet_text,language,tweet_created_at,retweet,favorite,hashtags,user_id,user_name,user,user_loc,user_follower,friends_count,media,tweet_url
116,1324498946820067334,#30DayMapChallenge Day4 Hexágonos - A partir d...,es,2020-11-05 23:48:51,2,7,"[{'text': '30DayMapChallenge', 'indices': [0, ...",28714934,Alonso Cortés,aloCrts,Mexico DF,125,412,http://pbs.twimg.com/media/EmGMMI1WkAMRY_j.jpg,https://t.co/AIj90BF9vH
117,1324498269557448712,#30DayMapChallenge Only just made it for Day 5...,en,2020-11-05 23:46:10,1,3,"[{'text': '30DayMapChallenge', 'indices': [0, ...",996371006519496705,Menna Bell,BellMenna,"Wales, United Kingdom",8,62,http://pbs.twimg.com/media/EmGPo_XXYAI153M.jpg,https://t.co/GWRHjjCt0Z
118,1324497349327134720,Day 5 #30DayMapChallenge is about #Blue what r...,en,2020-11-05 23:42:30,1,0,"[{'text': '30DayMapChallenge', 'indices': [6, ...",3290545829,Carmem Rodrigues 🏡😷🇧🇷🌍,CarmemMarx,"Belo Horizonte, Minas Gerais",181,632,http://pbs.twimg.com/media/EmGPT-aX0AMT6_F.jpg,https://t.co/sxelADgWay
119,1324493814615183363,#30DayMapChallenge 5 Blue @AlcaldiadeMed inter...,es,2020-11-05 23:28:27,4,11,"[{'text': '30DayMapChallenge', 'indices': [0, ...",1193871042558730241,vecinosdela80,vecinosdela80,,1251,3983,http://pbs.twimg.com/media/EmGME0RX0AomOFJ.jpg,https://t.co/OG0pIbEN1R
120,1324491260644728834,#30DayMapChallenge Day 5: Blue\nToday I used #...,en,2020-11-05 23:18:19,2,1,"[{'text': '30DayMapChallenge', 'indices': [0, ...",104213130,R.Richard,rorich,"Atlanta, GA",256,1312,http://pbs.twimg.com/media/EmGJv7AXYAAEmIo.jpg,https://t.co/sodGxP9KxW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1324141560770908160,#30DayMapChallenge Day 5 | Blue\n\nWhen ideas ...,en,2020-11-05 00:08:44,2,9,"[{'text': '30DayMapChallenge', 'indices': [0, ...",2614024302,Rob Radburn,robradburn,"Leicester, UK",3181,749,http://pbs.twimg.com/media/EmBLumxWkAMR4Ij.jpg,https://t.co/vww6deAh31
493,1324141334383439873,Day 4 of #30DayMapChallenge \n - Hexagones\nWo...,en,2020-11-05 00:07:50,1,4,"[{'text': '30DayMapChallenge', 'indices': [9, ...",2593554955,Kapazitätsdiode,varaktor,,24,182,http://pbs.twimg.com/media/EmBLJyiWoAMwkJP.jpg,https://t.co/DxQ6BCVi4w
494,1324141226837377024,"#30DayMapChallenge 5. Blue.\n\nBlue elevation,...",no,2020-11-05 00:07:24,1,9,"[{'text': '30DayMapChallenge', 'indices': [0, ...",44981447,Tore Halset,torehalset,"Stavanger, Norway",285,735,http://pbs.twimg.com/media/EmBLJljXYAURsus.jpg,https://t.co/SyEBAq59nw
495,1324140549817905154,"I first studied the ""Ring of Fire"" in an intro...",en,2020-11-05 00:04:43,1,5,"[{'text': '30DayMapChallenge', 'indices': [124...",133541065,Eric Goodwin,mr_ericgoodwin,"Center of Tennessee, Tennessee",381,586,http://pbs.twimg.com/media/EmBJqfJWoAEdX6G.jpg,https://t.co/v6v7FKPAoi


### late entry (only for day 2)

In [7]:
# df[df['tweet_text'].str.contains('points', case=False) & 
#    df['tweet_text'].str.contains('day 1', case=False)].to_csv("./data/{}_late.csv".format(date))

In [8]:
# df = df.drop(df[df['tweet_text'].str.contains('points', case=False) & 
#                 df['tweet_text'].str.contains('day 1', case=False)].index)

---

## Overview

In [16]:
df[['retweet','favorite', 'user_follower', 'friends_count']].describe()

,retweet,favorite,user_follower,friends_count
count,393.000000,393.000000,393.000000,393.000000
mean,3.249364,16.778626,1409.720102,662.134860
std,5.836978,36.911358,5689.031444,857.301559
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,4.000000,77.000000,158.000000
50%,2.000000,8.000000,280.000000,335.000000
75%,4.000000,18.000000,876.000000,780.000000
max,91.000000,624.000000,80333.000000,5454.000000


## Top favorited/retweeted `#30mapchallenges` post

In [81]:
def sort_map(df, sortby, num):
    df = df.sort_values(by='favorite', ascending=False).head(num)
    df['user'] = '**[@' + df['user'] +']('+ df['tweet_url']+')**'
    df['media'] = '![](' + df['media'] +')'
    df = df[['user', 'retweet', 'favorite', 'media']].set_index("user")
    return(tabulate(df, headers='keys', tablefmt='github'))

In [83]:
def tweet_shortcode(df, sortby, num):
    df = df.sort_values(by=sortby, ascending=False).head(num)
    df['shortcode'] = '{{< tweet ' + df['tweet_id'].apply(str) + ' >}}'
    df = df[['shortcode']]
    shortcode_ls = []
    for i in df['shortcode']:
        shortcode_ls.append(i)
    return shortcode_ls

In [84]:
with open('day{}_{}.md'.format(date[-2:], theme_lookup[date[-2:]]), 'w') as f:
    print(date +'...\n')
    print("Creating mardowkdown file ...")
    heading1 = "## Top favorited post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading1+ '\n')
    f.write(sort_map(df, 'favorite', 15)+ '\n \n')
    
    heading2 = "### Top retweeted post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading2+ '\n')
    f.write(sort_map(df, 'retweet', 15))

2020-11-05...

Creating mardowkdown file ...


In [85]:
with open('../../Blog/mywebsite/content/en/post/30daymap-{}-{}.md'.format(date[-2:], theme_lookup[date[-2:]]), 'w') as f:
    print(date +'...\n')
    print("Creating mardowkdown file ...")

    heading1 = "## Top favorited post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading1+ '\n')
    for s in tweet_shortcode(df, 'favorite', 15):
        f.write(s + '\n')
    
    f.write('\n')

    heading2 = "### Top retweeted post for day {} `#30mapchallenges ({})`".format(date[-2:], theme_lookup[date[-2:]])
    f.write(heading2+ '\n')
    for s in tweet_shortcode(df, 'retweet', 15):
        f.write(s + '\n')

2020-11-05...

Creating mardowkdown file ...


### Late entry

In [25]:
# df_late = pd.read_csv("./data/{}_late.csv".format(date))
# print("{}_late.csv".format(date))
# print("Add to day 1...")

2020-11-02_late.csv
Add to day 1...


In [27]:
# df_late_fav = df_late.sort_values(by='favorite', ascending=False).head(5)
# df_late_fav = df_late_fav[['user', 'retweet', 'favorite', 'media']]
# df_late_fav['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_fav['media'] = '![](' + df_late['media'] +')'

In [28]:
print(tabulate(df_late_fav, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 42 | **[@darenw](https://t.co/rZ2gtbJmI4)**          |         4 |         95 | ![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h.jpg) |
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 53 | **[@PratapVardhan](https://t.co/WTT8WM17qy)**   |         4 |         43 | ![](http://pbs.twimg.com/media/Elx5722WkAEOOMf.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 54 | **[@BZgeo](https://t.co/LavH2067O2)**           |         3 |         26 | ![](http://pbs.twimg.com/media/Elx0-fPXIAc9JGJ.png) |


In [29]:
# df_late_rt = df_late.sort_values(by='retweet', ascending=False).head(5)
# df_late_rt = df_late_rt[['user', 'retweet', 'favorite', 'media']]
# df_late_rt['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_rt['media'] = '![](' + df_late['media'] +')'

In [30]:
# print(tabulate(df_late_rt, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 24 | **[@neocartocnrs](https://t.co/1GcTGraT9T)**    |         8 |         14 | ![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 57 | **[@nurussadad](https://t.co/z4HaIfQMBi)**      |         4 |          7 | ![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT.png) |
|  7 | **[@geologyjesse](https://t.co/dR68mE4zhD)**    |         4 |         14 | ![](http://pbs.twimg.com/media/El1vroKWoAEozEk.jpg) |


### Send tweet via API

In [48]:
status = 'A github repo of monitoring tweets feeds with wonderful maps. Github: https://github.com/hn303/30DayMapChallenge-Bot'

In [45]:
imagePath = "screenshot/github.com_hn303_30DayMapChallenge-Bot.png"

In [49]:
api.update_status(status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 13:14:23 +0000 2020', 'id': 1323976890621792256, 'id_str': '1323976890621792256', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. **Github**:https://t.co/J1NHDOPpdT', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [73, 96]}]}, 'source': '<a href="https://www.landecon.cam.ac.uk/research/research-centres/lisa/members" rel="nofollow">Untitle_APP3</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1078706606488068096, 'id_str': '1078706606488068096', 'name': 'Stan', 'screen_name': 'Stan_CamLE', 'location': '', 'description': '', 'url': None, 'entit

In [46]:
api.update_with_media(imagePath, status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 12:58:56 +0000 2020', 'id': 1323973002543976450, 'id_str': '1323973002543976450', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. https://t.co/J1NHDOPpdT https://t.co/HwA2aVTax1', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [62, 85]}], 'media': [{'id': 1323972999717003266, 'id_str': '1323972999717003266', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'media_url_https': 'https://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'url': 'https://t.co/HwA2aVTax1', 'display_url': 'pic.twitter.com/HwA2aVTax1', 'expanded_url': 'https://twitter.com/Stan_CamLE/status/1323973002543976450/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 